In [14]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import seaborn as sns 


In [16]:
df=pd.read_csv("/Users/saikarthik/Desktop/Nexus-S1 Deakin /data.csv")

In [18]:
df.head()


,d3mIndex,timestamp,value,ground_truth
0,0,1472918400,0.234191,0
1,1,1472918700,0.236657,0
2,2,1472919000,0.230887,0
3,3,1472919300,0.217486,0
4,4,1472919600,0.223722,0


In [19]:
df.tail()

,d3mIndex,timestamp,value,ground_truth
8779,8779,1475552100,0.305849,0
8780,8780,1475552400,0.308899,0
8781,8781,1475552700,0.307782,0
8782,8782,1475553000,0.309060,0
8783,8783,1475553300,0.309851,0


In [20]:
df.shape

(8784, 4)

In [21]:
df["ground_truth"].value_counts()

ground_truth
0    8771
1      13
Name: count, dtype: int64